# Boltzmann Machines  
## Prepare Data

In [15]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [16]:
movies = pd.read_csv('ml-1m/movies.dat', sep='::', header=None, engine='python', encoding='latin-1')
users = pd.read_csv('ml-1m/users.dat', sep='::', header=None, engine='python', encoding='latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep='::', header=None, engine='python', encoding='latin-1')

In [17]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter='\t')
test_set = pd.read_csv('ml-100k/u1.test', delimiter='\t')
training_set.head()

,1,1.1,5,874965758
0,1,2,3,876893171
1,1,3,4,878542960
2,1,4,3,876893119
3,1,5,3,889751712
4,1,7,4,875071561


In [18]:
training_set = np.array(training_set, dtype='int')
test_set = np.array(test_set, dtype='int')

In [19]:
# get the number of users and movies
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))
print(nb_users)
print(nb_movies)

943
1682


In [20]:
# converting the data into a matrix with users in lines and movies in columns
def convert(data):
    new_data = []
    for id_user in range(1, nb_users+1):
        id_movies = data[:,1][data[:,0] == id_user]
        id_ratings = data[:,2][data[:,0] == id_user]
        ratings = np.zeros(nb_movies)
        ratings[id_movies-1] = id_ratings
        new_data.append(list(ratings))
    return new_data
            
training_set = convert(training_set)
test_set = convert(test_set)           

In [21]:
# converting data into Torch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [22]:
# converting ratings into binary ratings 1 (Liked) or 0 (Not Liked)
training_set[training_set == 0] = -1  #this indexing filtering only works with pytorch tensors
training_set[training_set < 3] = 0
training_set[training_set >= 3] = 1
test_set[test_set == 0] = -1
test_set[test_set < 3] = 0
test_set[test_set >= 3] = 1

## Build the RBM (Restricted Boltzmann Machine)

In [53]:
class RBM():
    
    def __init__(self, nv, nh):
        self.W = torch.randn(nh, nv)
        self.a = torch.randn(1, nh)
        self.b = torch.randn(1, nv)
        
    def sample_h(self, x):
        wx = torch.mm(x, self.W.t())
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    
    def sample_v(self, y):
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    
    def train(self, v0, vk, ph0, phk):
        self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)

In [71]:
nv = nb_movies
nh = 100
batch_size = 100

rbm = RBM(nv, nh)

## Training RBM

In [72]:
nb_epoch = 10

for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user:id_user+batch_size]
        v0 = training_set[id_user:id_user+batch_size]
        ph0, _ = rbm.sample_h(v0)
        for k in range(10):
            _, hk = rbm.sample_h(vk)
            _, vk = rbm.sample_v(hk)
            vk[v0 < 0] = v0[v0 < 0]
        phk, _ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0 >= 0] - vk[v0 >= 0]))
        s += 1. 
        print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: tensor(0.4658)
epoch: 1 loss: tensor(0.5451)
epoch: 1 loss: tensor(0.4845)
epoch: 1 loss: tensor(0.4441)
epoch: 1 loss: tensor(0.4133)
epoch: 1 loss: tensor(0.4000)
epoch: 1 loss: tensor(0.3801)
epoch: 1 loss: tensor(0.3745)
epoch: 1 loss: tensor(0.3621)
epoch: 2 loss: tensor(0.2138)
epoch: 2 loss: tensor(0.2200)
epoch: 2 loss: tensor(0.2173)
epoch: 2 loss: tensor(0.2186)
epoch: 2 loss: tensor(0.2237)
epoch: 2 loss: tensor(0.2335)
epoch: 2 loss: tensor(0.2359)
epoch: 2 loss: tensor(0.2398)
epoch: 2 loss: tensor(0.2416)
epoch: 3 loss: tensor(0.2488)
epoch: 3 loss: tensor(0.2401)
epoch: 3 loss: tensor(0.2397)
epoch: 3 loss: tensor(0.2361)
epoch: 3 loss: tensor(0.2379)
epoch: 3 loss: tensor(0.2450)
epoch: 3 loss: tensor(0.2461)
epoch: 3 loss: tensor(0.2499)
epoch: 3 loss: tensor(0.2508)
epoch: 4 loss: tensor(0.2528)
epoch: 4 loss: tensor(0.2445)
epoch: 4 loss: tensor(0.2386)
epoch: 4 loss: tensor(0.2338)
epoch: 4 loss: tensor(0.2371)
epoch: 4 loss: tensor(0.2443)
epoch: 4 l

## Testing RBM

In [75]:
test_loss = 0
s = 0.
for id_user in range(0, nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _, h = rbm.sample_h(v)
        _, v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1. 
print('test loss: '+str(test_loss/s))

test loss: tensor(0.2466)
